In [29]:
# imports
import numpy as np
import os

from numpy.random import default_rng
# stabilité du notebook d'une exécution à l'autre
random=default_rng(42) 

# jolies figures directement dans le notebook
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# où sauver les figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "workflowDS"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID) # le dossier doit exister

In [33]:
import pandas as pd

general_data = pd.read_csv('Donnees/general_data.csv')
employee_survey_data = pd.read_csv('Donnees/employee_survey_data.csv')
manager_survey_data = pd.read_csv('Donnees/manager_survey_data.csv')
in_time = pd.read_csv('Donnees/in_time.csv')
out_time = pd.read_csv('Donnees/out_time.csv')

In [34]:

def rename_column(dataset, name, position):
    tbl = dataset.columns.to_list()
    tbl[position] = name
    dataset.columns = tbl

rename_column(in_time, 'EmployeeID', 0)

# calculer les absence de chaque employé

# def get_absence(dataset):
#     absence = []
#     for i in range(len(dataset)):
#         absence.append(dataset.iloc[i].isna().sum())
#     return absence

# absence = get_absence(in_time)

# # ajouter la colonne absence au dataset

# in_time['absence'] = absence

in_time.head()


,",""2015-01-01"",""2015-01-02"",""2015-01-05"",""2015-01-06"",""2015-01-07"",""2015-01-08"",""2015-01-09"",""2015-01-12"",""2015-01-13"",""2015-01-14"",""2015-01-15"",""2015-01-16"",""2015-01-19"",""2015-01-20"",""2015-01-21"",""2015-01-22"",""2015-01-23"",""2015-01-26"",""2015-01-27"",""2015-01-28"",""2015-01-29"",""2015-01-30"",""2015-02-02"",""2015-02-03"",""2015-02-04"",""2015-02-05"",""2015-02-06"",""2015-02-09"",""2015-02-10"",""2015-02-11"",""2015-02-12"",""2015-02-13"",""2015-02-16"",""2015-02-17"",""2015-02-18"",""2015-02-19"",""2015-02-20"",""2015-02-23"",""2015-02-24"",""2015-02-25"",""2015-02-26"",""2015-02-27"",""2015-03-02"",""2015-03-03"",""2015-03-04"",""2015-03-05"",""2015-03-06"",""2015-03-09"",""2015-03-10"",""2015-03-11"",""2015-03-12"",""2015-03-13"",""2015-03-16"",""2015-03-17"",""2015-03-18"",""2015-03-19"",""2015-03-20"",""2015-03-23"",""2015-03-24"",""2015-03-25"",""2015-03-26"",""2015-03-27"",""2015-03-30"",""2015-03-31"",""2015-04-01"",""2015-04-02"",""2015-04-03"",""2015-04-06"",""2015-04-07"",""2015-04-08"",""2015-04-09"",""2015-04-10"",""2015-04-13"",""2015-04-14"",""2015-04-15"",""2015-04-16"",""2015-04-17"",""2015-04-20"",""2015-04-21"",""2015-04-22"",""2015-04-23"",""2015-04-24"",""2015-04-27"",""2015-04-28"",""2015-04-29"",""2015-04-30"",""2015-05-01"",""2015-05-04"",""2015-05-05"",""2015-05-06"",""2015-05-07"",""2015-05-08"",""2015-05-11"",""2015-05-12"",""2015-05-13"",""2015-05-14"",""2015-05-15"",""2015-05-18"",""2015-05-19"",""2015-05-20"",""2015-05-21"",""2015-05-22"",""2015-05-25"",""2015-05-26"",""2015-05-27"",""2015-05-28"",""2015-05-29"",""2015-06-01"",""2015-06-02"",""2015-06-03"",""2015-06-04"",""2015-06-05"",""2015-06-08"",""2015-06-09"",""2015-06-10"",""2015-06-11"",""2015-06-12"",""2015-06-15"",""2015-06-16"",""2015-06-17"",""2015-06-18"",""2015-06-19"",""2015-06-22"",""2015-06-23"",""2015-06-24"",""2015-06-25"",""2015-06-26"",""2015-06-29"",""2015-06-30"",""2015-07-01"",""2015-07-02"",""2015-07-03"",""2015-07-06"",""2015-07-07"",""2015-07-08"",""2015-07-09"",""2015-07-10"",""2015-07-13"",""2015-07-14"",""2015-07-15"",""2015-07-16"",""2015-07-17"",""2015-07-20"",""2015-07-21"",""2015-07-22"",""2015-07-23"",""2015-07-24"",""2015-07-27"",""2015-07-28"",""2015-07-29"",""2015-07-30"",""2015-07-31"",""2015-08-03"",""2015-08-04"",""2015-08-05"",""2015-08-06"",""2015-08-07"",""2015-08-10"",""2015-08-11"",""2015-08-12"",""2015-08-13"",""2015-08-14"",""2015-08-17"",""2015-08-18"",""2015-08-19"",""2015-08-20"",""2015-08-21"",""2015-08-24"",""2015-08-25"",""2015-08-26"",""2015-08-27"",""2015-08-28"",""2015-08-31"",""2015-09-01"",""2015-09-02"",""2015-09-03"",""2015-09-04"",""2015-09-07"",""2015-09-08"",""2015-09-09"",""2015-09-10"",""2015-09-11"",""2015-09-14"",""2015-09-15"",""2015-09-16"",""2015-09-17"",""2015-09-18"",""2015-09-21"",""2015-09-22"",""2015-09-23"",""2015-09-24"",""2015-09-25"",""2015-09-28"",""2015-09-29"",""2015-09-30"",""2015-10-01"",""2015-10-02"",""2015-10-05"",""2015-10-06"",""2015-10-07"",""2015-10-08"",""2015-10-09"",""2015-10-12"",""2015-10-13"",""2015-10-14"",""2015-10-15"",""2015-10-16"",""2015-10-19"",""2015-10-20"",""2015-10-21"",""2015-10-22"",""2015-10-23"",""2015-10-26"",""2015-10-27"",""2015-10-28"",""2015-10-29"",""2015-10-30"",""2015-11-02"",""2015-11-03"",""2015-11-04"",""2015-11-05"",""2015-11-06"",""2015-11-09"",""2015-11-10"",""2015-11-11"",""2015-11-12"",""2015-11-13"",""2015-11-16"",""2015-11-17"",""2015-11-18"",""2015-11-19"",""2015-11-20"",""2015-11-23"",""2015-11-24"",""2015-11-25"",""2015-11-26"",""2015-11-27"",""2015-11-30"",""2015-12-01"",""2015-12-02"",""2015-12-03"",""2015-12-04"",""2015-12-07"",""2015-12-08"",""2015-12-09"",""2015-12-10"",""2015-12-11"",""2015-12-14"",""2015-12-15"",""2015-12-16"",""2015-12-17"",""2015-12-18"",""2015-12-21"",""2015-12-22"",""2015-12-23"",""2015-12-24"",""2015-12-25"",""2015-12-28"",""2015-12-29"",""2015-12-30"",""2015-12-31"""
0,"1,NA,""2015-01-02 09:43:45"",""2015-01-05 10:08:4..."
1,"2,NA,""2015-01-02 10: